In [6]:
import os
import pandas as pd
import numpy as np
import math
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
results_dir = "../../symphony/backtest/backtests/results/optimization_results/"
files = [fname for fname in os.listdir(results_dir) if fname != '240_results.csv']
print(files)

['USDCHF240_results.csv', 'GBPUSD60_results.csv', '1440_results.csv', 'USDCAD240_results.csv', 'EURUSD240_results.csv', 'USDCHF60_results.csv', 'GBPUSD240_results.csv', 'NZDUSD240_results.csv', 'AUDUSD60_results.csv', 'EURUSD60_results.csv', 'USDCAD60_results.csv', 'AUDUSD240_results.csv', 'NZDUSD60_results.csv']


In [8]:
dfs = [pd.read_csv(results_dir + filename) for filename in files]
results_df = pd.concat(dfs)
results_df = results_df.drop_duplicates(subset=None, keep='first', inplace=False)
results_df.head()

,INSTRUMENT,TIMEFRAME,POSSIBLE_BUY_TRADES,POSSIBLE_SELL_TRADES,TOTAL_BUY_TRADES,TOTAL_SELL_TRADES,PROFITABLE_BUY_TRADES,PROFITABLE_SELL_TRADES,TOTAL_PROFIT,AVG_PROFIT,...,MAX_DRAWDOWN,CCI,CCII,ADX_SIMPLE,ADX_LOOKBACK,RSI_RANGE,CLOSE_ON_CONFLICT,TRAILING_STOP,1ATR_STOP,FIB_TP
0,USDCHF,240,46,73,42,51,11,25,20344.1280,218.75,...,87.471869,False,False,False,False,False,True,True,False,1.0
1,USDCHF,240,46,73,30,26,8,15,19189.3065,342.67,...,87.471018,False,False,False,False,True,True,True,False,1.0
2,USDCHF,240,46,73,30,26,8,15,19189.3065,342.67,...,87.471018,False,False,False,True,True,True,True,False,1.0
3,USDCHF,240,46,73,30,26,8,15,19189.3065,342.67,...,87.471018,False,False,True,True,True,True,True,False,1.0
4,USDCHF,240,46,73,30,26,8,15,19189.3065,342.67,...,87.471018,False,False,True,False,True,True,True,False,1.0


In [9]:
trading_cols = ["CCI", "CCII", "ADX_SIMPLE", "ADX_LOOKBACK", "RSI_RANGE", "CLOSE_ON_CONFLICT", "TRAILING_STOP", "1ATR_STOP", "FIB_TP"]
leave_out = ["TIMEFRAME", "PERC_TRADES_TAKEN", "TOTAL_PROFIT_IN_PIPS", "INSTRUMENT"]
backtest_cols = [col for col in results_df.columns if col not in trading_cols + leave_out]


In [10]:
profit_df = results_df[results_df["TOTAL_PROFIT"] > 0]
results_df.head(50)[backtest_cols + ["CCI", "CCII"]]

,POSSIBLE_BUY_TRADES,POSSIBLE_SELL_TRADES,TOTAL_BUY_TRADES,TOTAL_SELL_TRADES,PROFITABLE_BUY_TRADES,PROFITABLE_SELL_TRADES,TOTAL_PROFIT,AVG_PROFIT,PERC_PROFITABLE,SHARPE_RATIO,DRAWDOWN,MAX_DRAWDOWN,CCI,CCII
0,46,73,42,51,11,25,20344.1280,218.75,0.39,0.472741,11.853000,87.471869,False,False
1,46,73,30,26,8,15,19189.3065,342.67,0.41,0.480848,10.408771,87.471018,False,False
2,46,73,30,26,8,15,19189.3065,342.67,0.41,0.480848,10.408771,87.471018,False,False
3,46,73,30,26,8,15,19189.3065,342.67,0.41,0.480848,10.408771,87.471018,False,False
4,46,73,30,26,8,15,19189.3065,342.67,0.41,0.480848,10.408771,87.471018,False,False
5,46,73,39,46,10,23,16571.5601,194.96,0.39,0.424782,11.636605,87.471698,False,False
6,46,73,39,46,10,23,16571.5601,194.96,0.39,0.424782,11.636605,87.471698,False,False
7,46,73,39,46,10,23,16571.5601,194.96,0.39,0.424782,11.636605,87.471698,False,False
8,46,73,25,27,14,11,12483.3757,240.06,0.48,0.697565,12.211027,60.275457,False,False
9,46,73,25,27,14,11,12483.3757,240.06,0.48,0.697565,12.211027,60.275457,False,False


In [11]:
profit_df.columns


Index(['INSTRUMENT', 'TIMEFRAME', 'POSSIBLE_BUY_TRADES',
       'POSSIBLE_SELL_TRADES', 'TOTAL_BUY_TRADES', 'TOTAL_SELL_TRADES',
       'PROFITABLE_BUY_TRADES', 'PROFITABLE_SELL_TRADES', 'TOTAL_PROFIT',
       'AVG_PROFIT', 'PERC_PROFITABLE', 'PERC_TRADES_TAKEN', 'SHARPE_RATIO',
       'DRAWDOWN', 'MAX_DRAWDOWN', 'CCI', 'CCII', 'ADX_SIMPLE', 'ADX_LOOKBACK',
       'RSI_RANGE', 'CLOSE_ON_CONFLICT', 'TRAILING_STOP', '1ATR_STOP',
       'FIB_TP'],
      dtype='object')

In [12]:
print("Len before cleaning: {}".format(len(results_df)))
clean_df = results_df[results_df["PERC_TRADES_TAKEN"] > 0.0]
print("Len after cleaning: {}".format(len(clean_df)))

Len before cleaning: 4608
Len after cleaning: 4416


In [13]:
# EVENT / NONEVENT
total_unprofitable = len(results_df[results_df["TOTAL_PROFIT"] <= 10000])
total_profitable = len(results_df[results_df["TOTAL_PROFIT"] > 10000])
for col in trading_cols:
    num_unprofitable = len(results_df[(results_df[col] == True) & (results_df["TOTAL_PROFIT"] <= 10000)])
    num_profitable = len(results_df[(results_df[col] == True) & (results_df["TOTAL_PROFIT"] > 10000)])
    perc_event = (num_profitable / total_profitable)
    perc_nonevent = (num_unprofitable / total_unprofitable)
    woe = math.log(perc_event / perc_nonevent)
    iv = round((perc_event - perc_nonevent) * woe,3)
    print("Column: {} - WOE: {}, IV: {}".format(col, woe, iv))
    

Column: CCI - WOE: -2.6800278171500636, IV: 1.256
Column: CCII - WOE: -1.9864463260512613, IV: 0.862
Column: ADX_SIMPLE - WOE: -0.19033334537600835, IV: 0.017
Column: ADX_LOOKBACK - WOE: -0.19033334537600835, IV: 0.017
Column: RSI_RANGE - WOE: -0.19033334537600835, IV: 0.017
Column: CLOSE_ON_CONFLICT - WOE: 0.6271628826112109, IV: 0.272
Column: TRAILING_STOP - WOE: -0.10985424232133736, IV: 0.006
Column: 1ATR_STOP - WOE: 0.09909545241071419, IV: 0.005
Column: FIB_TP - WOE: 0.0, IV: 0.0


In [27]:
def result(iv):
    if iv < 0.02:
        return "Not useful"
    elif iv >= 0.02 and iv < 0.1:
        return "Weak predictor"
    elif iv >= 0.1 and iv < 0.3:
        return "Medium predictor"
    elif iv >= 0.2 and iv <0.5:
        return "Strong predictor"
    elif iv >= 0.5:
        return "Suspicious"
    else:
        return "Unknown"

In [28]:
# EVENT / NONEVENT
profit_level = 250
drawdown_level = 12.0
total_unprofitable = len(results_df[(results_df["AVG_PROFIT"] <= profit_level) | (results_df["MAX_DRAWDOWN"] > drawdown_level)])
total_profitable = len(results_df[(results_df["AVG_PROFIT"] > profit_level) & (results_df["MAX_DRAWDOWN"] <= drawdown_level)])
for col in trading_cols:
    num_unprofitable = len(results_df[(results_df[col] == True) & (results_df["AVG_PROFIT"] <= profit_level) | (results_df["MAX_DRAWDOWN"] > drawdown_level)])
    num_profitable = len(results_df[(results_df[col] == True) & (results_df["AVG_PROFIT"] > profit_level) & (results_df["MAX_DRAWDOWN"] <= drawdown_level)])
    perc_event = (num_profitable / total_profitable)
    perc_nonevent = (num_unprofitable / total_unprofitable)
    woe = math.log(perc_event / perc_nonevent)
    iv = round((perc_event - perc_nonevent) * woe,3)
    print("Column: {} - WOE: {}, IV: {}, Res: {}".format(col, woe, iv, result(iv)))
    

Column: CCI - WOE: -0.06473141714663075, IV: 0.003, Res: Not useful
Column: CCII - WOE: -0.8184021955391262, IV: 0.338, Res: Strong predictor
Column: ADX_SIMPLE - WOE: -0.40506198836589463, IV: 0.103, Res: Medium predictor
Column: ADX_LOOKBACK - WOE: -0.40506198836589463, IV: 0.103, Res: Medium predictor
Column: RSI_RANGE - WOE: -0.3430961135222304, IV: 0.08, Res: Weak predictor
Column: CLOSE_ON_CONFLICT - WOE: -0.47008060180505623, IV: 0.134, Res: Medium predictor
Column: TRAILING_STOP - WOE: -0.22138776867859916, IV: 0.035, Res: Weak predictor
Column: 1ATR_STOP - WOE: -0.45042640485820756, IV: 0.128, Res: Medium predictor
Column: FIB_TP - WOE: 0.0, IV: 0.0, Res: Not useful


In [20]:
# EVENT / NONEVENT
profit_level = 2500
total_unprofitable = len(results_df[(results_df["TOTAL_PROFIT"] <= profit_level) ])
total_profitable = len(results_df[(results_df["TOTAL_PROFIT"] > profit_level)])
for col in trading_cols:
    num_unprofitable = len(results_df[(results_df[col] == True) & (results_df["TOTAL_PROFIT"] <= profit_level)])
    num_profitable = len(results_df[(results_df[col] == True) & (results_df["TOTAL_PROFIT"] > profit_level)])
    perc_event = (num_profitable / total_profitable)
    perc_nonevent = (num_unprofitable / total_unprofitable)
    woe = math.log(perc_event / perc_nonevent)
    iv = round((perc_event - perc_nonevent) * woe,3)
    print("Column: {} - WOE: {}, IV: {}, Res: {}".format(col, woe, iv, result(iv)))
    

Column: CCI - WOE: -0.1833820527364665, IV: 0.016, Res: Not useful
Column: CCII - WOE: 0.03982069995123231, IV: 0.001, Res: Not useful
Column: ADX_SIMPLE - WOE: -0.03311839792053719, IV: 0.001, Res: Not useful
Column: ADX_LOOKBACK - WOE: -0.03311839792053719, IV: 0.001, Res: Not useful
Column: RSI_RANGE - WOE: -0.18781451696740967, IV: 0.016, Res: Not useful
Column: CLOSE_ON_CONFLICT - WOE: 0.01340380987715456, IV: 0.0, Res: Not useful
Column: TRAILING_STOP - WOE: -0.21473680064752912, IV: 0.021, Res: Weak predictor
Column: 1ATR_STOP - WOE: -0.001925729639025637, IV: 0.0, Res: Not useful
Column: FIB_TP - WOE: 0.0, IV: 0.0, Res: Not useful
